# Launch CANape with Python through ctypes Interface

## Example Goals:

- Load ```CANapAPI64.dll``` with ctypes.
- Open CANape.
- Exit CANape.

## References:

- [```ctypes``` - A foreign function for Python](https://docs.python.org/3/library/ctypes.html)
- [CANape ASAM-MCD3 Interface Version Application Note AN-AMC-1-103](http://docplayer.net/56065428-Canape-asam-mcd3-interface-version-application-note-an-amc-1-103.html)
- ```CANapAPI.h``` (C:\Program Files\Vector CANape 19\CANapeAPI\CANapAPI.h)
        Description:
        |    ASAP3-Interface
        |    Version    : 1.0 28-AUG-2000
        |-----------------------------------------------------------------------------
        | Copyright (c) by Vector Informatik GmbH.  All rights reserved.

In [10]:
import os
import ctypes
CANapAPI_dll = os.path.abspath(r"C:\Program Files\Vector CANape 19.0\CANapeAPI\CANapAPI64.dll")
# Load the library and assign it to dll.
dll = ctypes.windll.LoadLibrary(CANapAPI_dll)

Start to 'port' `CANapAPI.h`.

This creates a handle for reference in some functions.

```struct tAsap3Hdl;```

In [11]:
class tAsap3Hdl(ctypes.Structure):
    pass

```typedef struct tAsap3Hdl *TAsap3Hdl;```

In [12]:
TAsap3Hdl = ctypes.POINTER(tAsap3Hdl)

Create a handle.

In [13]:
hdl=TAsap3Hdl()

# Asap3Init

Returns a handle to be used in subsequent function calls. CANape will be started (if not actually running)
 
- hdl \output Asap3Handle for further access to this interface.
- responseTimeout \input Maximum response time
- workingDir \input Sets CANape  working directory By default the project file CANape.INI is saved at the working directory used for the CANape session. 
  If you want to load a different project file, please append the project file name to 'workingDir'.
- fifoSize \input Total size of FIFO used for data acquisition,    
  i.e. number of FIFO entries which can be read out using Asap3GetNextSample(). Each FIFO entry includes at most ACQ_MAX_VALUES=128 measurement values.
 *
- debugMode \input if this is true call CANape in 'normal' screen size instead of 'minimized'.
  if it is false call CANape in 'minimized' mode.

```
extern bool Asap3Init(
    TAsap3Hdl * hdl,
    unsigned long responseTimeout,
    const char *workingDir,
    unsigned long fifoSize,
    bool debugMode);
```

Define variables in Python

In [14]:
# Maximum response time (ms)
responseTimeout = 10000 # 10 seconds
# CANape requires absolute path.
workingDir = os.path.abspath(".")
fifoSize = 8192
debugMode = True

> By default functions are assumed to return the C int type. Other return types can be specified by setting the restype attribute of the function object. - [Return Types](https://docs.python.org/3/library/ctypes.html#return-types)

> It is possible to specify the required argument types of functions exported from DLLs by setting the argtypes attribute. - [`ctypes` documentation](https://docs.python.org/3/library/ctypes.html#specifying-the-required-argument-types-function-prototypes)

In [15]:
dll.Asap3Init.restype = ctypes.c_bool
dll.Asap3Init.argtypes = (
    ctypes.POINTER(TAsap3Hdl),
    ctypes.c_ulong,
    ctypes.c_char_p,
    ctypes.c_ulong,
    ctypes.c_bool,
)

Cast them to `ctypes` types. This is hypothetically covered by argtypes, but in practice `c_char_p` is not.

```ArgumentError: argument 3: <class 'TypeError'>: wrong type```

In [17]:
c_responseTimeout = ctypes.c_ulong()
c_workingDir = ctypes.c_char_p(workingDir.encode("UTF-8"))
c_fifoSize = ctypes.c_ulong(fifoSize)
c_debugMode = ctypes.c_bool(debugMode)

Launch CANape:

In [18]:
dll.Asap3Init(
    ctypes.byref(hdl),
    c_responseTimeout,
    c_workingDir,
    c_fifoSize,
    c_debugMode,
)

True

Exit CANape:

In [23]:
dll.Asap3Exit.argtypes= (TAsap3Hdl, )
dll.Asap3Exit.restype=ctypes.c_bool

result = dll.Asap3Exit(hdl)
result

True

# Appendices
Required Software Installed:

- Python 3.x
- Jupyter Notebook
- Vector CANape 19.0 Demo